In [1]:
import os, sys
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier

os.sys.path.append(Path().cwd().parents[0].as_posix())

In [2]:
from src.MyModule.utils import *
from src.MyModule.privacy_test import MembershipInference, Reidentification, AttributeDisclosure

# from src.MyModule.privacy_test import *

 this is project path : /home/wonseok/projects/2022_DATA_SYNTHESIS/young_age_privacy/notebooks 


In [3]:
age = 50
config = load_config()

In [4]:
class Argument :
    
    def __init__(self, age, random_seed):
        self.age = age
        self.random_seed = random_seed
        
args = Argument(50, 0)

In [5]:
projectPath = Path(config['project_path'])

In [7]:
preprocessedDict = get_machine_learning_data(projectPath, age, 0)

original = preprocessedDict['original']
holdout = preprocessedDict['holdout']
syntheticDict = preprocessedDict['synthetic_dict']

## Modeling Testing with KNN

In [30]:
x1 = syntheticDict['epsilon0']
y1 = np.array([1]*x1.shape[0])

x2 = holdout.copy()
y2 = np.array([0]*x2.shape[0])

In [36]:
model1 = KNeighborsClassifier()
model2 = KNeighborsClassifier()

In [37]:
model1.fit(x1, y1)
model2.fit(x2, y2)

KNeighborsClassifier()

In [38]:
indice1 = model1.kneighbors(original, n_neighbors=1, return_distance=False)
indice2 = model2.kneighbors(original, n_neighbors=1, return_distance=False)

In [40]:
indice1.shape, indice2.shape

((626, 1), (626, 1))

In [45]:
np.array([[1,2,3]]) == np.array([[1,2,1]])

array([[ True,  True, False]])

## Membership Inference

In [8]:
syntheticSample = syntheticDict['epsilon0']

In [9]:
membInf = MembershipInference(original = original,
                    holdout = holdout,
                    synthetic = syntheticSample)

In [10]:
membInf.infer()

0.17192212777625446

## Reidentification

In [11]:
syntheticSample = syntheticDict['epsilon0']

In [12]:
reIden = Reidentification(original=original,
                          holdout = holdout,
                          synthetic=syntheticSample,
                          feature_split=0.5)

In [13]:
reIden.infer()

(array([0.01118211]), array([0.]))

## Attribute

In [8]:
syntheticSample = syntheticDict['epsilon0']

In [19]:
attDisc = AttributeDisclosure(original = original,
                              synthetic = syntheticSample,
                              sensitive_features=['BSPT_SEX_CD','BSPT_IDGN_AGE'])

In [20]:
answer = attDisc.infer()

--------------------
--------------------------infering for label : BSPT_SEX_CD----------------------
------------------------inferring for original data ------------------------
------------------------inferring for label : BSPT_SEX_CD-----------------------
------------------------inferring for synthetic data -----------------------
--------------------------infering for label : BSPT_IDGN_AGE----------------------
------------------------inferring for original data ------------------------
------------------------inferring for label : BSPT_IDGN_AGE-----------------------
------------------------inferring for synthetic data -----------------------


In [ ]:
 reform = {(outerKey, innerKey): values for outerKey, innerDict in dictionary.items() for innerKey, values in innerDict.items()}

In [63]:
a = {(outerkey, innerkey) : [round(values, 3)] for outerkey, innerDict in zip(['baseline','synthetic'], answer) for innerkey, values in innerDict.items()}

In [78]:
b = pd.DataFrame(a).T.rename(columns = {0:'epsilon 0'})
c = pd.DataFrame(a).T.rename(columns = {0:'epsilon 0.1'})

In [88]:
pd.concat([b,c], axis=1)

epsilon 0  epsilon 0.1
baseline  BSPT_SEX_CD        0.541        0.541
          BSPT_IDGN_AGE      0.082        0.082
synthetic BSPT_SEX_CD        0.613        0.613
          BSPT_IDGN_AGE      0.116        0.116